# Development of Emotion and Reasoning in the General Speeches of the United Nations: A text-based machine learning approach
## Additional Analysis: Different individualised Stopwords - Model Traning, Calculation Centroids & Speech Scoring

### Description

In the replication of Gennaro & Ash, their stopwords list was used. Since stopwords should be adapted for a different corpus, this script tests whether using a custom stopword list changes the results. 


In [2]:
# == Import libraries for data processing and NLP ==

import os
import gensim
from pathlib import Path
from gensim.models import Word2Vec
import numpy as np

from nltk.tokenize import sent_tokenize
from random import shuffle
import nltk
from gensim.utils import simple_preprocess
import joblib
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from scipy.spatial.distance import cosine
import glob
import spacy
from multiprocessing import Pool, freeze_support
import pandas as pd


# == Initialize NLP Tools ===

translator = str.maketrans('', '', punctuation) 
tagger = nltk.perceptron.PerceptronTagger()
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
stemmer = SnowballStemmer("english")

# == Set Working Directory and create folder structure ==

# --- Set base path to project root ---
base_path = Path.cwd().parents[2]  # project root
print(f"Project root set to: {base_path}")

# === Define Folder Paths ===

# Folders were already created in the script 0_data_creation
data_c = base_path / "data"
data_temp = data_c / "temp"
data_freq = data_c / "freq"
data_dict = data_c / "dictionaries"
data_sent = data_c / "sentences"
data_preprocessed = data_c / "preprocessed"
data_results = data_c / "results"
data_models = data_c / "models" 
data_stopwords = data_c / "stopwords"

# Upload ressources
stopwords = joblib.load(data_stopwords / "ind_stopwords.pkl")         
word_counts = joblib.load(data_freq / "ind_stopwords_word_counts.pkl")
word_counts_weighted = joblib.load(data_freq / "ind_stopwords_word_counts_weighted.pkl")
affect_dic = joblib.load(data_dict / 'dictionary_affect.pkl')
cognition_dic = joblib.load(data_dict / 'dictionary_cognition.pkl') 

Project root set to: C:\Users\sarah\Downloads\TESTRUN


In [3]:
os.chdir(data_temp)
cleaned_files = [
    str(data_temp / 'clean_speeches_indexed1.pkl'),
    str(data_temp / 'clean_speeches_indexed2.pkl'),
    str(data_temp / 'clean_speeches_indexed3.pkl'),
    str(data_temp / 'clean_speeches_indexed4.pkl')
]

___

### Sentence Split

In [6]:
os.chdir(data_temp)

# Function to split cleaned speeches (clean_speeches) into sentences, tokenize, clean, tag, stem, filter, and save them.

def extract_sentences(dataname):
    data = joblib.load(dataname)
    data = [a[1] for a in data]  # keep only text, no id

    sentences = []
    for doc in data:
        sentences += sent_tokenize(doc)  # use nltk's sent_tokenize here

    sentences = [item for item in sentences if len(item.split()) > 1]
    sentences = [gensim.utils.simple_preprocess(item) for item in sentences]

    sentences = [[a for a in s if not a.isdigit()] for s in sentences]
    sentences = [[a for a in s if len(a) > 2] for s in sentences]

    sentences = [tagger.tag(s) for s in sentences]
    sentences = [[i[0] for i in s if i[1].startswith(('N', 'V', 'J'))] for s in sentences]

    sentences = [[stemmer.stem(i) for i in s] for s in sentences]
    sentences = [[a for a in s if a not in stopwords] for s in sentences]
    sentences = [[a for a in s if word_counts[a] >= 10] for s in sentences]

    dropped_count = sum(1 for s in sentences if len(s) <= 1)
    print(f"Number of very short sentences being dropped: {dropped_count}")

    sentences = [s for s in sentences if len(s) > 1]
    shuffle(sentences)

    lab = dataname.replace('clean_speeches_', 'ind_stopwords_sentences_').replace('_.pkl', '.pkl')
    print(f'{dataname} processed')
    joblib.dump(sentences, lab)
    
    unique_tokens = set(token for s in sentences for token in s)
    print(f"Unique tokens: {len(unique_tokens)}")

    # Print preview of first 5 processed sentences
    print("Example sentences (first 5):")
    for s in sentences[:5]:
        print(s)


    print(f'{lab} saved')

for fname in cleaned_files:
    extract_sentences(fname)

Number of very short sentences being dropped: 7149
C:\Users\sarah\Downloads\TESTRUN\data\temp\clean_speeches_indexed1.pkl processed
Unique tokens: 11068
Example sentences (first 5):
['review', 'charter', 'unit', 'overdu', 'guilti', 'imperi']
['meet', 'para']
['help', 'believ', 'popul', 'contribut', 'small']
['particular', 'caus', 'indign', 'aggress', 'cambodia']
['portugues', 'govern', 'understand', 'born', 'thrive', 'grow', 'win']
C:\Users\sarah\Downloads\TESTRUN\data\temp\ind_stopwords_sentences_indexed1.pkl saved
Number of very short sentences being dropped: 3960
C:\Users\sarah\Downloads\TESTRUN\data\temp\clean_speeches_indexed2.pkl processed
Unique tokens: 11524
Example sentences (first 5):
['tunisia', 'conceal', 'surpris', 'toler', 'attitud', 'solidar', 'principl', 'taken', 'western', 'world', 'israel', 'violat', 'principl', 'charter', 'defianc', 'unit']
['allow', 'prospect', 'collect', 'suffer', 'sake', 'subordin', 'interest']
['term', 'draft', 'new', 'intern', 'order', 'establis

In [7]:
# Pick the first file to see how the sentence split looks like
file_path = os.path.join(data_temp, 'ind_stopwords_sentences_indexed1.pkl')

sentences = joblib.load(file_path)

print("Example sentences (first 5):")
for s in sentences[:5]:
    print(s)

Example sentences (first 5):
['review', 'charter', 'unit', 'overdu', 'guilti', 'imperi']
['meet', 'para']
['help', 'believ', 'popul', 'contribut', 'small']
['particular', 'caus', 'indign', 'aggress', 'cambodia']
['portugues', 'govern', 'understand', 'born', 'thrive', 'grow', 'win']


In [8]:
sentences_files = [
    os.path.join(data_temp, 'ind_stopwords_sentences_indexed1.pkl'),
    os.path.join(data_temp, 'ind_stopwords_sentences_indexed2.pkl'),
    os.path.join(data_temp, 'ind_stopwords_sentences_indexed3.pkl'),
    os.path.join(data_temp, 'ind_stopwords_sentences_indexed4.pkl')
]

In [9]:
# == Get sum of unique tokens ==
all_unique_tokens = set()
total_token_count = 0

for dataname in sentences_files:
    data = joblib.load(dataname)
    for sentence in data:
        all_unique_tokens.update(sentence)
        total_token_count += len(sentence)

print(f"Total unique tokens across all files: {len(all_unique_tokens)}")
print(f"Total number of tokens): {total_token_count}")

sentences_files = [
    'ind_stopwords_sentences_indexed1.pkl',
    'ind_stopwords_sentences_indexed2.pkl',
    'ind_stopwords_sentences_indexed3.pkl',
    'ind_stopwords_sentences_indexed4.pkl']

Total unique tokens across all files: 12475
Total number of tokens): 12798938


---

### Train Word2Vec

In [12]:
os.chdir(data_temp)

In [13]:
dataset = []

for dataname in sentences_files:  
    data = joblib.load(dataname)
    dataset.extend(data) 

# === Model training ===
w2v = Word2Vec(
    sentences=dataset,    
    vector_size=300,      # Dimension of the vector
    window=8,             # Context window size
    min_count=10,         # Minimum word count
    workers=1,            # Number of threads
    sample=1e-3,          # Downsample setting for frequent words
    epochs=10,            # Number of iterations over the corpus
    seed = 19
)


w2v.wv.fill_norms() 

# Save model
data_models.mkdir(parents=True, exist_ok=True) 
w2v.save(str(data_models / 'ind_stopwords_w2v-vectors_8_300.pkl'))

In [14]:
w2v = Word2Vec.load(str(data_models / "ind_stopwords_w2v-vectors_8_300.pkl"))
word_vectors = w2v.wv

---

### Calculate Centroids

In [17]:
# == Calculation ==
def findcentroid(text, model):
    vecs = [model.wv[w] * word_counts_weighted[w] 
        for w in text 
        if w in model.wv and w in word_counts_weighted]
    vecs = [v for v in vecs if len(v) > 0]
    centroid = np.mean(vecs, axis=0)
    #centroid = centroid.reshape(1, -1)
    return centroid


affect_centroid = findcentroid(affect_dic, w2v)
cog_centroid = findcentroid(cognition_dic, w2v)

os.chdir(data_c)
joblib.dump(affect_centroid, 'centroids/ind_stopwords_affect_centroid.pkl')
joblib.dump(cog_centroid, 'centroids/ind_stopwords_cog_centroid.pkl')

# == Overview Vectors ==
print("Affect centroid vector:\n", affect_centroid)
print("\nCognition centroid vector:\n", cog_centroid)

# Shape and stats
print("\nShape of affect centroid:", affect_centroid.shape)
print("Shape of cognition centroid:", cog_centroid.shape)

Affect centroid vector:
 [ 1.75194472e-01 -3.40102389e-02 -6.08769543e-02 -1.43784806e-01
  1.85608432e-01  3.20071787e-01 -2.70114839e-01 -5.32627925e-02
 -3.40738088e-01 -1.31559104e-01 -1.70375071e-02  1.70824993e-02
 -1.04281004e-04 -2.24624917e-01 -2.91625679e-01  1.97496787e-01
 -1.44104943e-01 -3.51681322e-01 -2.02808276e-01  8.49855393e-02
 -1.99525580e-01  5.58591366e-01 -6.63657859e-02  1.09758824e-01
 -1.30453333e-01  1.09566040e-01  7.35619850e-03 -2.87883058e-02
 -8.90789274e-03 -4.27111663e-04 -1.02948323e-02  4.45364490e-02
 -8.15507248e-02 -1.37557238e-01 -7.04562515e-02  4.20600213e-02
 -1.99324727e-01 -1.18877487e-02  7.39056543e-02  2.93308310e-02
  7.06296712e-02  3.01391371e-02 -9.18107107e-02 -1.57915920e-01
 -2.40626894e-02  3.31540890e-02 -4.04579239e-03 -1.26371533e-01
  1.52846873e-01  1.11754779e-02 -2.93758195e-02 -1.59001481e-02
 -5.41262031e-02 -2.05493793e-01  3.38682503e-01  8.31910148e-02
  1.69820175e-01 -4.90108244e-02 -1.50323346e-01 -1.06529810e-01


---

### Emotionality Scoring

In [20]:
os.chdir(data_preprocessed)

# Load preprocessed speech data 

preprocessed_final_files = [
   joblib.load(os.path.join(data_preprocessed, 'ind_stopwords_preprocessed_speeches_indexed1_final.pkl')),
   joblib.load(os.path.join(data_preprocessed, 'ind_stopwords_preprocessed_speeches_indexed2_final.pkl')),
   joblib.load(os.path.join(data_preprocessed, 'ind_stopwords_preprocessed_speeches_indexed3_final.pkl')),
   joblib.load(os.path.join(data_preprocessed, 'ind_stopwords_preprocessed_speeches_indexed4_final.pkl'))
]

In [21]:
# Function to compute weighted document vectors and derive affective/cognitive distances and scores

def documentvecweight(lista):
    out = []
    lista = [i for i in lista if len(i[1]) > 0]
    for s in lista:
        # Compute weighted word vectors for each token present in the Word2Vec model
        vecs = [w2v.wv[w] * word_counts_weighted[w] for w in s[1] if w in w2v.wv]
        if len(vecs) == 0:
            a = np.nan
            c = np.nan
            score = np.nan
        else:
            # Compute mean vector for each speech
            v = np.mean(vecs, axis=0)
             # Cosine distance to affective centroid
            a = cosine(v, affect_centroid)
            # Cosine distance to cognitive centroid
            c = cosine(v, cog_centroid)
            score = (1 + 1 - a) / (1 + 1 - c)
        out.append([s[0], a, c, score])
    return out


def main_function(file_path, idx):
    dataset = joblib.load(file_path)
    data = documentvecweight(dataset)
    lab = os.path.join(data_results, f'ind_stopwords_distances_main_{idx}.pkl')
    joblib.dump(data, lab)

# Main loop: process all preprocessed speech files
def main():
    files = [
        os.path.join(data_preprocessed, f'ind_stopwords_preprocessed_speeches_indexed{i+1}_final.pkl') #Changed!
        for i in range(4)
    ]
    for i, f in enumerate(files, start=1):
        main_function(f, i)

if __name__ == "__main__":
    main()

# Merge all distance files into one df
DATA_temp = [os.path.join(data_results, f'ind_stopwords_distances_main_{i+1}.pkl') for i in range(4)]

tot = []
for dataname in DATA_temp:
    d = joblib.load(dataname)
    tot += d

tot_df = pd.DataFrame(tot, columns=['filename', 'affect_d', 'cognition_d', 'score'])
joblib.dump(tot_df, os.path.join(data_results, 'ind_stopwords_distances_10epochs.pkl'))

['C:\\Users\\sarah\\Downloads\\TESTRUN\\data\\results\\ind_stopwords_distances_10epochs.pkl']

In [22]:
print(tot_df.head())

          filename  affect_d  cognition_d     score
0  ARG_01_1946.txt  1.113648     0.728064  0.696853
1  AUS_01_1946.txt  1.548899     0.873628  0.400490
2  BEL_01_1946.txt  1.588663     0.961471  0.396077
3  BLR_01_1946.txt  1.194433     0.944733  0.763378
4  BOL_01_1946.txt  0.925866     0.701981  0.827518


In [24]:
# Load un_corpus_cleaned_final and merge with tot_df by filename 
un_corpus_cleaned_final = pd.read_csv(os.path.join(data_preprocessed, "un_corpus_cleaned_final_ind_stopwords.csv"), sep=';', encoding='utf-8') 
un_corpus_scored = un_corpus_cleaned_final.merge(tot_df, on="filename", how="left")

joblib.dump(un_corpus_scored, os.path.join(data_results, "ind_stopwords_un_corpus_scored.pkl"))

un_corpus_scored.to_csv(
    os.path.join(data_results, "ind_stopwords_un_corpus_scored.csv"),
    sep=';', 
    index=False, 
    encoding='utf-8'
)

In [25]:
print(un_corpus_scored)

              filename                                             speech  \
0      ARG_01_1946.txt  At the resumption of the first session of the ...   
1      AUS_01_1946.txt  The General Assembly of the United Nations is ...   
2      BEL_01_1946.txt  The principal organs of the United Nations hav...   
3      BLR_01_1946.txt  As more than a year has elapsed since the Unit...   
4      BOL_01_1946.txt  Coming to this platform where so many distingu...   
...                ...                                                ...   
10947  WSM_79_2024.txt  Excellencies, I extend my congratulations to H...   
10948  YEM_79_2024.txt  Your Majesties, Excellencies, and Highnesses, ...   
10949  ZAF_79_2024.txt  President of the 79th Session of the UN Genera...   
10950  ZMB_79_2024.txt  YOUR EXCELLENCY PHILEMON YANG, PRESIDENT OF TH...   
10951  ZWE_79_2024.txt  Your Excellency, Mr. Philemon Yang, President ...   

      country_code  year  country_name  speech_length_words  \
0           

In [26]:
# Count where affect_d is NaN
nan_count = un_corpus_scored['affect_d'].isna().sum()

print("Count where affect_d is NaN:", nan_count)

Count where affect_d is NaN: 0
